In [ ]:
#| default_exp config

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import pandas as pd

In [ ]:
#| export
REF_COLOR = "k"
FEMALE_COLOR = "C1"
MALE_COLOR = "C0"
ALL_COLOR = "C5"

GLUC_COLOR = "C0"
FOOD_COLOR = "C1"

DATASETS_PATH = '/home/ec2-user/studies/'
POPULATION_DATASET = 'Population_Characteristics'

In [ ]:
#| export

def generate_synthetic_data(n: int = 1000) -> pd.DataFrame:
    """
    Generates a sample DataFrame containing age, gender, and value data.

    Args:
        n: The number of rows in the generated DataFrame.

    Returns:
        A pandas DataFrame with columns 'age', 'gender', and 'val'.
    """
    pids = np.arange(n)
    # Set start and end dates
    start_date = pd.Timestamp('2020-01-01')
    end_date = pd.Timestamp('now')
    dates = pd.to_datetime(pd.to_datetime(np.random.uniform(start_date.value, end_date.value, n).astype(np.int64)).date)  
    ages = np.random.uniform(35, 73, size=n)
    genders = np.random.choice([0, 1], size=n)
    vals = np.random.normal(30 + 1 * ages + 40 * genders, 20, size=n)
    
    data = pd.DataFrame(data={"participant_id":pids,"date_of_research_stage": dates,"age_at_research_stage": ages, "sex": genders, "val1": vals}).set_index("participant_id")
    data["val2"] = data["val1"]*0.3 + 0.5*np.random.normal(0,50) + 0.2*10*data["sex"]
    return data

In [ ]:
#| export

def generate_synthetic_data_like(df: pd.DataFrame, n: int = 1000, random_seed: int = 42) -> pd.DataFrame:
    """
    Generate a sample DataFrame containing the same columns as `df`, but with random data.

    Args:
    
        df: The DataFrame whose columns should be used.
        n: The number of rows in the generated DataFrame.

    Returns:
        A pandas DataFrame with the same columns as `df`.
    """
    np.random.seed(random_seed)
    pids = np.arange(n)
    if n > len(df):
        replace = True
    else:
        replace = False

    null = df.reset_index().apply(lambda x: x.sample(frac=1).values)\
        .sample(n=n, replace=replace).assign(participant_id=pids)\
        .set_index(df.index.names)

    def is_path_string(x):
        return isinstance(x, str) and (x.count('/') > 1)

    # handle specific columns
    null.loc[:, null.applymap(is_path_string).mean() > 0.5] = '/path/to/file'
    if ('collection_timestamp' in null.columns) and ('collection_date' in null.columns):
        null['collection_date'] = null['collection_timestamp'].dt.date

    return null

In [ ]:
data = generate_synthetic_data()
data.head()

,date_of_research_stage,age_at_research_stage,sex,val1,val2
participant_id,,,,,
0,2021-05-29,69.788555,1,162.270810,82.273910
1,2020-12-13,36.289947,1,125.240476,71.164810
2,2021-12-30,61.501970,1,116.044417,68.405992
3,2020-06-15,46.299262,0,84.440308,56.924759
4,2020-12-14,70.127055,1,120.693921,69.800843


In [ ]:
generate_synthetic_data_like(data.head(), n=5)

,date_of_research_stage,age_at_research_stage,sex,val1,val2
participant_id,,,,,
0,2021-12-30,46.299262,1,84.440308,82.273910
1,2020-06-15,36.289947,1,162.270810,69.800843
2,2020-12-14,61.501970,1,120.693921,71.164810
3,2020-12-13,69.788555,1,116.044417,68.405992
4,2021-05-29,70.127055,0,125.240476,56.924759


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()